In [2]:
import sys
import pandas as pd
import copy
import numpy as np
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', None)

In [2]:
sys.path.append('/Users/benjaminblalock/cmu_faculty_student_net/')

# Faculty

In [152]:
core_faculty = pd.read_csv('../data/scraped/core_faculty.csv')

affiliated_faculty = pd.read_csv('../data/scraped/affiliated_faculty.csv')
affiliated_faculty['research_interests'] = np.nan

related_faculty = pd.read_csv('../data/scraped/related_faculty.csv')
related_faculty['research_interests'] = np.nan

faculty_master = pd.concat([core_faculty, affiliated_faculty, related_faculty])
faculty_master_list = faculty_master.name.tolist()

In [153]:
faculty_master.head()

,entity_type,faculty_type,name,title,research_interests
0,faculty,core,Maria-Florina Balcan,"Professor, Computer Science & Machine Learning...","Foundations of Machine Learning, Data Driven A..."
1,faculty,core,Ziv Bar-Joseph,"Professor, Computational Biology & Machine Lea...","Computational Biology, Graphical Models, Time ..."
2,faculty,core,Tianqi Chen,"(Joining Fall 2020) Assistant Professor, Machi...","Machine Learning Systems, Large-scale machine ..."
3,faculty,core,William Cohen,"(ON LEAVE) Professor, Machine Learning & Langu...",NaN
4,faculty,core,Christos Faloutsos,"Professor, Computer Science, School of Compute...","Data Mining, Social Networks, Anomaly Detection"


# Students

## Current

In [203]:
import re
from utils import get_top_matches

current_phd_students = pd.read_csv('../data/scraped/current_phd_students.csv')
current_phd_students['advisor'] = current_phd_students.advisor.str.split(',')
current_phd_students = current_phd_students.explode('advisor')
current_phd_students['advisor'] = current_phd_students.advisor.apply(lambda x: re.sub('(\(.+?\))', '', str(x).strip()))
current_phd_students.loc[current_phd_students.advisor == 'nan', 'advisor'] = None
current_phd_students['match'] = (current_phd_students
                                 .advisor
                                 .apply(lambda x: [tup[1] for tup 
                                                   in get_top_matches([str(x)], faculty_master_list, normalize_func=lambda i: i) 
                                                   if (tup[2]>=.9)
                                                  ]
                                       )
                                )
current_phd_students = current_phd_students.explode('match')

advisors_not_in_faculty_master = current_phd_students.loc[(current_phd_students.match.isna()) & (current_phd_students.advisor.notna()), ['advisor']]
advisors_not_in_faculty_master.columns=['name']
advisors_not_in_faculty_master['entity_type'] = 'faculty'
advisors_not_in_faculty_master['faculty_type'] = 'unknown'
advisors_not_in_faculty_master['title'] = np.nan
advisors_not_in_faculty_master['research_interests'] = np.nan
advisors_not_in_faculty_master = advisors_not_in_faculty_master[['entity_type', 'faculty_type', 'name', 'title', 'research_interests']]

faculty_master = pd.concat([faculty_master, advisors_not_in_faculty_master])

current_phd_students['match'] = current_phd_students.match.combine_first(current_phd_students.advisor)

current_students_node_frame = current_phd_students.loc[:,['name', 'education', 'research_interests']].drop_duplicates()
current_students_edge_frame = current_phd_students.loc[:,['name', 'match']].drop_duplicates()
current_students_edge_frame.columns = ['student', 'faculty']
current_students_edge_frame['edge_type'] = 'advisor'


In [205]:
current_students_node_frame

,name,education,research_interests
0,Nil-Jana Akpinar,"University of Freiburg, Master of Science in M...","Statistical methodology, ML in high-stakes dec..."
1,Arish Alreja,"Georgia Institute of Technology, BS & MS in El...","Biological and Computer Vision, Unsupervised L..."
2,Maruan Al-Shedivat,"KAUST, MS in Computer Science, Moscow State Un...","Statistical Learning Theory, Learning Represen..."
3,Siddharth Ancha,"University of Toronto, MS in Computer Science","MCMC Inference, Deep Learning, Computer Vision"
4,Ifigeneia Apostolopoulou,"University of Thessaly at Volos, BS in Electri...","Pattern Mining, Statistical Machine Learning"
5,Shaojie Bai,"Carnegie Mellon University, BS in Computer Sci...","Deep Learning, Sequence Modeling, Optimizations"
6,Anna Bair,"Massachusetts Institute of Technology, SB & ME...","Theoretical Machine Learning, Fairness, Comple..."
7,Arundhati Banerjee,"Indian Institute of Technology, Kharagpur, BTe...","Theoretical Machine Learning, Deep Learning, D..."
8,Jonathon Byrd,"Morehead State University, BS in Computer Science","ML on graph and molecule data, transfer learni..."
9,Sebastian Caldas,"Universidad de los Andes, BS in Systems & Comp...",Machine learning for healthcare and the develo...


In [206]:
current_students_edge_frame

,student,faculty,edge_type
0,Nil-Jana Akpinar,Alexandra Chouldechova,advisor
0,Nil-Jana Akpinar,Zachary Lipton,advisor
1,Arish Alreja,Max G'Sell,advisor
1,Arish Alreja,Avniel S. Ghuman,advisor
2,Maruan Al-Shedivat,Eric Xing,advisor
3,Siddharth Ancha,David Held,advisor
3,Siddharth Ancha,Srinivasa Narasimhan,advisor
4,Ifigeneia Apostolopoulou,Artur Dubrawski,advisor
5,Shaojie Bai,Zico Kolter,advisor
6,Anna Bair,None,advisor


## Alumni

In [12]:
alumni_phd = pd.read_csv('../data/scraped/alumni_phd.csv')
alumni_phd.head()

,name,entity_type
0,Alnur Ali,alumni
1,Andrew Arnold,alumni
2,Pranjal Awasthi,alumni
3,Will Bishop,alumni
4,Byron Boots,alumni


In [16]:
dissertation_abstracts_matches = pd.read_csv('../data/scraped/dissertation_abstracts_matches.csv')
dissertation_abstracts_matches.head()

,alumni_match,faculty_match,faculty_role
0,Mary McGlohon,Christos Faloutsos,Chair
1,Jurij Leskovec,Christos Faloutsos,Chair
2,Zhenzhen Kou,William Cohen,Chair
3,Brent Bryan,Jeff Schneider,Chair
4,Jurij Leskovec,Christos Faloutsos,Chair


In [15]:
dissertation_pdf_matches = pd.read_csv('../data/scraped/dissertation_pdf_matches.csv')
dissertation_pdf_matches.sort_values('alumni_match')

,alumni_match,faculty_match,faculty_role
21,Aaditya Ramdas,Aaditya Ramdas,Non-chair
20,Aaditya Ramdas,Ryan Tibshirani,Non-chair
19,Aaditya Ramdas,Aarti Singh,Non-chair
18,Aaditya Ramdas,Larry Wasserman,Chair
17,Aaditya Ramdas,Aaditya Ramdas,Non-chair
16,Aaditya Ramdas,Ryan Tibshirani,Non-chair
15,Aaditya Ramdas,Aarti Singh,Non-chair
14,Aaditya Ramdas,Larry Wasserman,Chair
35,Ahmed Hefny,Eric Xing,Non-chair
34,Ahmed Hefny,Geoffrey Gordon,Chair


In [207]:
dissertation_abstracts_matches['source'] = 'abstract_page'
dissertation_pdf_matches['source'] = 'dissertation_pdf'
matches = (pd.concat([dissertation_abstracts_matches, dissertation_pdf_matches])
           .groupby([col for col in dissertation_abstracts_matches.columns 
                     if col not in ['source', 'faculty_role']
                    ]
                   )
           .agg(faculty_roles = ('faculty_role' ,  'unique'),
                sources = ('source' ,  'unique')
               )
           .reset_index()
          )

matches['chair'] = matches.faculty_roles.apply(lambda x: 1 if 'Chair' in x else 0)
matches['faculty_match_num_roles'] = matches.faculty_roles.apply(lambda x: len(x))
matches['faculty_match_num_sources'] = matches.sources.apply(lambda x: len(x))

alumni_counts = (copy.deepcopy(matches.reset_index())
                 .groupby(['alumni_match'])
                 .agg(num_chair_matches = ('chair', 'sum'),
                      num_faculty_matches = ('faculty_match', 'nunique')
                     )
                 .reset_index()
                )

review_frame = (pd.merge(alumni_counts, matches, how='inner', on='alumni_match')
                .sort_values(['num_chair_matches', 'num_faculty_matches', 'alumni_match', 'chair', 'faculty_match_num_sources', 'faculty_match_num_roles', 'faculty_match'],
                             ascending=[False, False, True, False, False, True, True]
                            )
                .reset_index(drop=True)
               )

review_frame.index.names = ['review_priority']
review_frame['correct_match'] = 1
review_frame['correct_role'] = review_frame.faculty_roles.apply(lambda x: 'Advisor' if 'Chair' in x else 'Committee')
review_frame

,alumni_match,num_chair_matches,num_faculty_matches,faculty_match,faculty_roles,sources,chair,faculty_match_num_roles,faculty_match_num_sources,correct_match,correct_role
review_priority,,,,,,,,,,,
0,Pradeep Ravikumar,3,11,Eric Xing,[Chair],"[abstract_page, dissertation_pdf]",1,1,2,1,Advisor
1,Pradeep Ravikumar,3,11,Seyoung Kim,[Chair],"[abstract_page, dissertation_pdf]",1,1,2,1,Advisor
2,Pradeep Ravikumar,3,11,Pradeep Ravikumar,"[Chair, Non-chair]","[abstract_page, dissertation_pdf]",1,2,2,1,Advisor
3,Pradeep Ravikumar,3,11,Ruslan Salakhutdinov,[Non-chair],"[abstract_page, dissertation_pdf]",0,1,2,1,Committee
4,Pradeep Ravikumar,3,11,Barnabás Póczos,[Non-chair],[abstract_page],0,1,1,1,Committee
5,Pradeep Ravikumar,3,11,Clark Glymour,[Non-chair],[abstract_page],0,1,1,1,Committee
6,Pradeep Ravikumar,3,11,Geoffrey Gordon,[Non-chair],[abstract_page],0,1,1,1,Committee
7,Pradeep Ravikumar,3,11,Kun Zhang,[Non-chair],[abstract_page],0,1,1,1,Committee
8,Pradeep Ravikumar,3,11,Manuela Veloso,[Non-chair],[abstract_page],0,1,1,1,Committee


#### (after human revision)

In [3]:
alumni_faculty_edges = review_frame.loc[review_frame.correct_match == 1, ['alumni_match', 'faculty_match', 'correct_role']].reset_index(drop=True)
alumni_faculty_edges.columns = ['student', 'faculty', 'edge_type']
alumni_faculty_edges

NameError: name 'review_frame' is not defined

### Co-occurence networks

In [3]:
student_faculty_connections = pd.read_csv('../data/transformed/student_faculty_relationships.csv')

In [4]:
def cooccurrence_edgelist(df, on):
    entity = 'faculty' if 'student' in on else 'student'
    edgelist = pd.merge(df, df, on=on, how='inner')
    edgelist = edgelist[edgelist[entity+'_x'] != edgelist[entity+'_y']]
    edgelist['pair'] = edgelist[[entity+'_x', entity+'_y']].values.tolist()
    edgelist['pair'] = edgelist['pair'].apply(lambda x: sorted(x))
    edgelist[['source', 'target']] = pd.DataFrame(edgelist.pair.tolist(), index=edgelist.index)
    edgelist['relationship'] = edgelist.apply(lambda x: 'Co-Advised' if (x['edge_type_x']=='Advisor') and (x.edge_type_y=='Advisor') else 'Co-Committee', axis=1)
    edgelist = edgelist.groupby(['source', 'target', 'relationship']).size().reset_index(name='weight')
    return edgelist

In [5]:
faculty_cooccurrence = cooccurrence_edgelist(student_faculty_connections, on='student')
student_cooccurrence = cooccurrence_edgelist(student_faculty_connections, on='faculty')
student_cooccurrences =[student_cooccurrence.relationship == 'Co-Advised']

In [6]:
student_cooccurrence.shape

(2315, 4)

In [7]:
student_cooccurrence[student_cooccurrence.relationship == 'Co-Advised'].shape

(712, 4)

In [8]:
faculty_master = pd.read_csv('../data/transformed/faculty_master.csv')
faculty_master['label'] = faculty_master['id']
faculty_master['parent'] = faculty_master['faculty_type']
faculty_master = faculty_master.drop_duplicates()
faculty_master.head()

,id,entity_type,faculty_type,title,research_interests,label,parent
0,Maria-Florina Balcan,faculty,core,"Professor, Computer Science & Machine Learning...","Foundations of Machine Learning, Data Driven A...",Maria-Florina Balcan,core
1,Ziv Bar-Joseph,faculty,core,"Professor, Computational Biology & Machine Lea...","Computational Biology, Graphical Models, Time ...",Ziv Bar-Joseph,core
2,Tianqi Chen,faculty,core,"(Joining Fall 2020) Assistant Professor, Machi...","Machine Learning Systems, Large-scale machine ...",Tianqi Chen,core
3,William Cohen,faculty,core,"(ON LEAVE) Professor, Machine Learning & Langu...",NaN,William Cohen,core
4,Christos Faloutsos,faculty,core,"Professor, Computer Science, School of Compute...","Data Mining, Social Networks, Anomaly Detection",Christos Faloutsos,core


In [9]:
faculty_master.shape

(111, 7)

In [10]:
len(G.nodes)

NameError: name 'G' is not defined

In [34]:
import networkx as nx

from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({'figure.figsize': (15, 10)})

# get reproducible results
import random
import numpy as np
random.seed(123)
np.random.seed(123)

color_scale = ['rgb(228,26,28)','rgb(55,126,184)',
               'rgb(77,175,74)','rgb(152,78,163)',
               'rgb(255,127,0)','rgb(255,255,51)',
               'rgb(166,86,40)'
              ]

def get_color(i):
    return color_scale[i] if i < len(color_scale) else 'rbg(0,0,0)'  

# TODO: There seems to be some edges dropping? from 1287718 to 1007793
def get_nx_graph(edge_file, node_file, create_using=nx.DiGraph):
    g = nx.from_pandas_edgelist(
        edge_file,
        'source',
        'target',
        edge_attr=['weight', 'relationship'],
        create_using=create_using
    )
    
    node_attr = node_file.set_index('id').to_dict('index')
    nx.set_node_attributes(g, node_attr)
    return g


def set_node_degree(G, name='degree'):
    degree = G.degree
    for v in G:
        G.nodes[v][name] = degree[v]
        
        
def set_node_pagerank(G, name='pagerank', **kwargs):
    pr = nx.pagerank(G, **kwargs)
    for v in G:
        G.nodes[v][name] = pr[v]
    
        
def set_node_community(G, name='community'):
        communities = sorted(nx.community.greedy_modularity_communities(G), key=len, reverse=True)
        print('there are '+ str(len(communities)) + ' communities')
        
        result = nx.community.girvan_newman(G)
        communities = next(result)
        print('there are '+ str(len(communities)) + ' communities')
        
        '''Add community to node attributes'''
        for c, v_c in enumerate(communities):
            for v in v_c:
                # Add 1 to save 0 for external edges
                G.nodes[v][name] = c + 1
                G.nodes[v][name+'_color'] = get_color(c + 1)

                
def set_edge_community(G, community_name='community'):
    '''Find internal edges and add their community to their attributes'''
    for v, w, in G.edges:
        if G.nodes[v][community_name] == G.nodes[w][community_name]:
            # Internal edge, mark with community
            G.edges[v, w][community_name] = G.nodes[v][community_name]
            G.edges[v, w][community_name+'_color'] = G.nodes[v][community_name+'_color']
        else:
            # External edge, mark as 0
            G.edges[v, w][community_name] = 0
            G.edges[v, w][community_name+'_color'] = 'grey'
    
def add_zero_degree_nodes(g, node_df):
    node_attr = node_df.set_index('id').to_dict('index')
    for node in node_attr:
        if node not in g:
            g.add_node(node)
            attr = node_attr[node]
            attr['degree']=0
            attr['pagerank']=0
            attr['community']=0
            attr['community_color']='rbg(1,1,1)'
            nx.set_node_attributes(g, {node:attr})

    return g
    
def create_node_edge_frame(G):
    node_attr = {node: G.nodes[node] for node in G.nodes}
    edge_attr = {edge: G.edges[edge[0], edge[1]] for edge in G.edges}
    
    node_frame = pd.DataFrame.from_dict(node_attr, orient='index').rename_axis('id').reset_index()
    edge_frame = pd.DataFrame.from_dict(edge_attr, orient='index').rename_axis(['source', 'target']).reset_index()
    
    return node_frame, edge_frame


# Create Network
G = get_nx_graph(faculty_cooccurrence, faculty_master, create_using=nx.Graph)
            
# Set node and edge communities
set_node_degree(G)
set_node_pagerank(G)
set_node_community(G)
set_edge_community(G)

print(len(G.nodes))
G = add_zero_degree_nodes(G, faculty_master)
print(len(G.nodes))

# Create node and edge_frame
node_frame, edge_frame = create_node_edge_frame(G)

there are 4 communities
there are 4 communities
58
111


In [35]:
node_frame.shape

(111, 11)

In [29]:
node_frame.head()

,id,entity_type,faculty_type,title,research_interests,label,parent,degree,pagerank,community,community_color
0,Aaditya Ramdas,faculty,core,"Assistant Professor, Statistics & Machine Lear...","Multiple Hypothesis Testing, Sequential Estima...",Aaditya Ramdas,core,1,0.003711,1,"rgb(55,126,184)"
1,Aarti Singh,faculty,core,"Associate Professor, Machine Learning, School ...",NaN,Aarti Singh,core,14,0.037037,1,"rgb(55,126,184)"
2,Barnabás Póczos,faculty,core,(ON LEAVE) Associate Professor Machine Learnin...,"Machine Learning in Scientific Applications, O...",Barnabás Póczos,core,14,0.036096,1,"rgb(55,126,184)"
3,Eric Xing,faculty,core,"Professor, Machine Learning & Language Technol...",NaN,Eric Xing,core,18,0.042823,1,"rgb(55,126,184)"
4,Jeff Schneider,faculty,core,"Research Professor, Robotics Institute, School...","Bayesian Optimization, Reinforcement Learning,...",Jeff Schneider,core,13,0.025463,1,"rgb(55,126,184)"


In [13]:
edge_frame.head()

,source,target,weight,relationship,community,community_color
0,Aaditya Ramdas,Aarti Singh,2,Co-Advised,1,"rgb(55,126,184)"
1,Aarti Singh,Barnabás Póczos,4,Co-Advised,1,"rgb(55,126,184)"
2,Aarti Singh,Eric Xing,4,Co-Committee,1,"rgb(55,126,184)"
3,Aarti Singh,Jeff Schneider,4,Co-Committee,1,"rgb(55,126,184)"
4,Aarti Singh,Larry Wasserman,8,Co-Committee,1,"rgb(55,126,184)"


In [20]:
pd.set_option('mode.chained_assignment', None)
def scale_num(array, a, b): 
    scaled = (b - a) * ((array - np.min(array)) / (np.max(array) - np.min(array))) + a
    if scaled is not None:
        return scaled 
    else:
        return a
    

def format_cyto_nodes(node_df, 
                      label='id',
                      classes='cyto_node',
                      parent=None,
                      size_by=None, 
                      min_size=20,
                      max_size=70,
                      opacity_by=None, 
                      min_opacity=0.3,
                      max_opacity=0.8,
                     ):
    node_df['label'] = node_df[label]
    
    if parent:
        node_df.loc[:,'parent'] = node_df[parent]
    if size_by:
        node_df['size'] = scale_num(node_df[size_by], min_size, max_size)
    if opacity_by:
        node_df['opacity'] = scale_num(node_df[opacity_by], min_opacity, max_opacity)
        
    nodes = node_df.rename(columns={label:'id'}).to_dict('index')
    return [{'data': nodes[index], 'classes': classes} for index in nodes]


def format_cyto_edges(edge_df, 
                      label=None,
                      classes='cyto_edge',
                      size_by=None, 
                      min_size=4,
                      max_size=7,
                      opacity_by=None, 
                      min_opacity=0.3,
                      max_opacity=0.6,
                     ):
    if size_by:
        edge_df['width'] = scale_num(edge_df['weight'], min_size, max_size)
    if opacity_by:
        edge_df['opacity'] = scale_num(edge_df['weight'], min_opacity, max_opacity)
    
    edges = edge_df.to_dict('index')
    return [{'data': edges[index], 'classes': classes} for index in edges]

# faculty_master = pd.merge(node_frame, 
#                           pd.concat([edge_frame.loc[:, ['source']].rename(columns={'source':'id'}),
#                                      edge_frame.loc[:, ['target']].rename(columns={'target':'id'}),
#                                     ]
#                                    )
#                           .drop_duplicates(),
#                           on='id',
#                           how='inner'
#                           )


faculty_root_nodes = format_cyto_nodes(node_frame[['entity_type']].drop_duplicates(),
                                       classes='faculty_root_node',
                                       label='entity_type'
                                       )

faculty_type_nodes = format_cyto_nodes(node_frame[['entity_type', 'faculty_type']].drop_duplicates(),
                                       parent='entity_type',
                                       classes='faculty_type_node',
                                       label='faculty_type'
                                       )

faculty_nodes = format_cyto_nodes(node_frame,
                                  parent='faculty_type',
                                  classes='faculty_node',
                                  size_by='degree',
                                  opacity_by='degree'
                                  )

faculty_co_advise_relations = format_cyto_edges(edge_frame[edge_frame.relationship == 'Co-Advised'],
                                                classes='co_advised_edge',
                                                size_by='weight',
                                                min_size=6,
                                                max_size=9,
                                                opacity_by='weight',
                                                min_opacity=0.4,
                                                max_opacity=0.7,
                                                )

faculty_co_committee_relations = format_cyto_edges(edge_frame[edge_frame.relationship == 'Co-Committee'],
                                                   classes='co_committee_edge',
                                                   size_by='weight',
                                                   min_size=3,
                                                   max_size=6,
                                                   opacity_by='weight',
                                                   min_opacity=0.2,
                                                   max_opacity=0.6,
                                                   )

In [22]:
import dash
import dash_cytoscape as cyto
import dash_html_components as html
# from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output

cyto.load_extra_layouts()

app = dash.Dash(__name__)

default_stylesheet = [
                {
                    'selector': '.faculty_root_node',
                    'style': {'content': 'data(label)',
                              'font-size': '50px',
                              'text-transform': 'uppercase',
                              'compound-sizing-wrt-labels': 'include',
                              }
                },
                {
                    'selector': '.faculty_type_node',
                    'style': {'content': 'data(label)',
                              'font-size': '30px',
                              'text-transform': 'uppercase',
                              'compound-sizing-wrt-labels': 'include',
                              }
                },
                {
                    'selector': '.faculty_node',
                    'style': {'content': 'data(label)',
                              'width': 'data(size)',
                              'height': 'data(size)',
                              'background-opacity': 'data(opacity)',
                              'background-color': 'data(community_color)'
                              }
                },
                {
                    'selector': '.co_advised_edge',
                    'style': {'line-style': 'solid',
                              'width': 'data(width)',
                              'opacity': 'data(opacity)',
                              'curve-style': 'bezier',
                              'line-color': 'blue'
                              }
                },
                {
                    'selector': '.co_committee_edge',
                    'style': {'line-style': 'dashed',
                              'width': 'data(width)',
                              'opacity': 'data(opacity)',
                              'curve-style': 'bezier',
                              'line-color': 'grey'
                              }
                }
            ]

app.layout = html.Div([
    cyto.Cytoscape(
        id='cmu_net',
        layout={
                'name': 'cose-bilkent',
                'animationEasing': 'ease-out',
                ## 'draft', 'default' or 'proof" 
                ## - 'draft' fast cooling rate 
                ## - 'default' moderate cooling rate 
                ## - "proof" slow cooling rate
                'quality': 'default',
                ## Whether to include labels in node dimensions. Useful for avoiding label overlap
                'nodeDimensionsIncludeLabels': 'true',
                ## number of ticks per frame; higher is faster but more jerky
                'refresh': 30,
                ## Whether to fit the network view after when done
                'fit': 'true',
                ## Padding on fit
                'padding': 100,
                ## Whether to enable incremental mode
                'randomize': 'False',
                ## Node repulsion (non overlapping) multiplier
                'nodeRepulsion': 100,
                ## Ideal (intra-graph) edge length
                'idealEdgeLength': 200,
                ## Divisor to compute edge forces
                'edgeElasticity': 0.5,
                ## Nesting factor (multiplier) to compute ideal edge length for inter-graph edges
                'nestingFactor': 0.1,
                ## Maximum number of iterations to perform
                'numIter': 2500,
                ## Whether to tile disconnected nodes
                'tile': 'true',
                ## Type of layout animation. The option set is {'during', 'end', false}
                'animate': 'end',
                ## Duration for animate:end
                'animationDuration': 500,
                ## Amount of vertical space to put between degree zero nodes during tiling (can also be a function)
                'tilingPaddingVertical': 10,
                ## Amount of horizontal space to put between degree zero nodes during tiling (can also be a function)
                'tilingPaddingHorizontal': 10,
                ## Gravity force (constant)
                'gravity': 0.25,
                ## Gravity range (constant) for compounds
                'gravityRangeCompound': 1.5,
                ## Gravity force (constant) for compounds
                'gravityCompound': 1.0,
                ## Gravity range (constant)
                'gravityRange': 3.8,
                ## Initial cooling factor for incremental layout
                'initialEnergyOnIncremental': 0.5
            },
            style={'width': '100%', 'height': '1500px'},
            stylesheet=default_stylesheet,
        elements=[
            *faculty_root_nodes,
            *faculty_type_nodes,
            *faculty_nodes,
            *faculty_co_advise_relations,
            *faculty_co_committee_relations
        ]
    )
])

@app.callback(Output('cmu_net', 'stylesheet'),
              [Input('cmu_net', 'tapNode')]
             )
def generate_stylesheet(node):
    print(len(node) if node else 'shit')
    if (not node) or (node['classes'] != 'faculty_node'):
        return default_stylesheet

    stylesheet = [
        {
            'selector': '.faculty_root_node',
            'style': {'content': 'data(label)',
                      'font-size': '50px',
                      'text-transform': 'uppercase',
                      'compound-sizing-wrt-labels': 'include',
                      }
        },
        {
            'selector': '.faculty_type_node',
            'style': {'content': 'data(label)',
                      'font-size': '30px',
                      'text-transform': 'uppercase',
                      'compound-sizing-wrt-labels': 'include',
                      }
        },
        {
            'selector': '.faculty_node',
            'style': {'width': 'data(size)',
                      'height': 'data(size)',
                      'background-opacity': 0.1,
                      }
        },
        {
            'selector': '.co_advised_edge',
            'style': {'line-style': 'solid',
                      'width': 'data(width)',
                      'opacity': 0.1,
                      'curve-style': 'bezier',
                      'line-color': 'blue'
                      }
        },
        {
            'selector': '.co_committee_edge',
            'style': {'line-style': 'dashed',
                      'width': 'data(width)',
                      'opacity': 0.1,
                      'curve-style': 'bezier',
                      'line-color': 'grey'
                      }
        },
        {
            "selector": '.faculty_node[id = "{}"]'.format(node['data']['id']),
            "style": {
                'background-color': '#B10DC9',
                "border-color": "purple",
                "border-width": 2,
                "border-opacity": 1,
                "background-opacity": 0.4,
                "label": "data(label)",
                "text-opacity": 1,
                "font-size": '26px',
                'z-index': 9999
            }
        }
    ]
            
        

    for edge in node['edgesData']:
        if edge['source'] == node['data']['id']:
            stylesheet.append({
                "selector": '.faculty_node[id = "{}"]'.format(edge['target']),
                "style": {
                    'background-color': 'blue',
                    'background-opacity': 0.4,
                    "border-color": "blue",
                    "border-width": 2,
                    "border-opacity": 1,
                    "label": "data(label)",
                    "text-opacity": 1,
                    "font-size": '26px'
                }
            })
            stylesheet.append({
                "selector": 'edge[id= "{}"]'.format(edge['id']),
                "style": {
                    "line-color": 'purple',
                    'opacity': 0.4,
                }
            })

        if edge['target'] == node['data']['id']:
            stylesheet.append({
                "selector": '.faculty_node[id = "{}"]'.format(edge['source']),
                "style": {
                    'background-color': 'blue',
                    'background-opacity': 0.4,
                    "border-color": "blue",
                    "border-width": 2,
                    "border-opacity": 1,
                    "label": "data(label)",
                    "text-opacity": 1,
                    "font-size": '26px'
                }
            })
            stylesheet.append({
                "selector": 'edge[id= "{}"]'.format(edge['id']),
                "style": {
                    "line-color": 'purple',
                    'opacity': 0.4,
                }
            })

    return stylesheet
    
app.run_server(use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Oct/2020 21:49:23] "GET /_reload-hash HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2020 21:49:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2020 21:49:23] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_8_2m1601034074.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2020 21:49:23] "GET /_dash-component-suites/dash_renderer/react@16.v1_8_2m1601034074.13.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2020 21:49:23] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_8_2m1601034074.13.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2020 21:49:24] "GET /_dash-component-suites/dash_renderer/dash_renderer.v1_8_2m1601034099.min.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2020 21:49:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2020 21:49:26] "GET /_dash-component-suites/dash_cytoscape/dash_cytoscape_extra.v0_2_0m1594671794.min.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2020 21:49:26] "GET /_dash-dependen

shit


127.0.0.1 - - [06/Oct/2020 21:49:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2020 21:49:28] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js.map HTTP/1.1" 200 -


shit


In [120]:
faculty_root_nodes

[{'data': {'id': 'faculty', 'label': 'faculty'},
  'classes': 'faculty_root_node'}]

shit


In [ ]:
faculty_type_nodes